In [1]:
!pip install numpy pandas matplotlib tensorflow keras


In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [2]:
data = pd.read_csv("C:\\Users\\hp\\Downloads\\Stock_data.csv")

# Sélectionner la colonne du prix de clôture (ajuster en fonction de vos données)
fermeture = data["Close"]


In [3]:
def preparation_donnees(data, seq_length):
  scaler = MinMaxScaler(feature_range=(0, 1))

  # Convert the Series to a NumPy array for reshaping
  data_array = fermeture.to_numpy()

  # Reshape is not necessary here for sequence creation (commented out)
  # data_reshape = data_array.reshape(-1, 2)  # Example reshape (optional)

  sequences = []
  for i in range(len(data_array) - seq_length):
    sequences.append(data_array[i:i + seq_length])

  X = np.array([seq[:-1] for seq in sequences])
  y = np.array([seq[-1] for seq in sequences])

  # Reshape X for LSTM
  X = X.reshape((X.shape[0], X.shape[1], 1))
  return X, y, scaler

seq_length = 30
X, y, scaler = preparation_donnees(fermeture, seq_length)


In [4]:
# Modèle séquentiel
model = Sequential()

# Couche LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))

# Couche dense de sortie
model.add(Dense(units=1))

# Compilation du modèle
model.compile(loss="mse", optimizer="adam")

# Entraînement du modèle
model.fit(X, y, epochs=100, batch_size=32)


C:\Users\hp\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - loss: 22944.2188
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 20988.3770
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 20268.3105
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 18782.3398
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 18429.7520
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 16835.1504
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 16625.6426
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 15855.5781
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 14872.7539
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 15007.2646
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 13752.9316
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 13516.4219
Epoch 13/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 13179.4951
Epoch 14/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 12146.2363
E

In [6]:
if os.path.exists('modele_entreine.h5'):
  try:
    model = load_model('modele_entreine.h5')
    # ... (use the loaded model for prediction)
  except OSError as e:
    print(f"Error loading model: {e}")
else:
  print("Model file 'modele_entreine.h5' not found. Please train the model first.")

Model file 'modele_entreine.h5' not found. Please train the model first.


In [33]:
# ... (previous code for data preparation)

# Ajuster la séquence d'entrée
derniers_prix = fermeture[-seq_length:]

# Convertir la série en tableau NumPy avant de reshaper
derniers_prix_array = derniers_prix.to_numpy()

# Reshape le tableau NumPy
derniers_prix_reshape = derniers_prix_array.reshape(-1, 1)

# Fit le scaler sur les données (avant la transformation)
scaler.fit(fermeture.to_numpy().reshape(-1, 1))  # Ajuster si nécessaire

# Transformer les derniers prix avec le scaler
derniers_prix_scale = scaler.transform(derniers_prix_reshape)

# ... (reste du code pour la prédiction et l'affichage du prix)

In [35]:
# Importer le modèle pour la prédiction
from tensorflow.keras.models import load_model

# Charger le modèle entraîné
model = load_model('modele_entreine.h5')

# Prédire le prochain prix
prix_predit = model.predict(derniers_prix_scale.reshape(1, seq_length, 1))

# Inverser la mise à l'échelle pour obtenir le prix réel prédit
prix_predit_inverse = scaler.inverse_transform(prix_predit)

# Afficher le prix prédit
print("Prochain prix prédit:", prix_predit_inverse[0][0])


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'modele_entreine.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Model file 'modele_entreine.h5' not found. Please train the model first.
